select stations

In [ ]:
import os
import pandas as pd

# ==============================
# 1️⃣ read nc station ids
# ==============================

data_dir = "/mnt/datawaha/hyex/wkkong/data/CAMELSH/timeseries/Data/CAMELSH/timeseries" #change to your path

# .nc file
nc_files = [f for f in os.listdir(data_dir) if f.endswith(".nc")]


nc_station_ids = [f.replace(".nc", "") for f in nc_files]

print("Number of nc stations:", len(nc_station_ids))


# ==============================
# 2️⃣ static attributes csv
# ==============================

attr_path = "/mnt/datawaha/hyex/wkkong/data/CAMELSH/attributes/attributes_gageii_BasinID.csv"

station = pd.read_csv(attr_path)

# ==============================
# 3️⃣ create station_id by zero-padding STAID to 8 digits
# ==============================

station["station_id"] = station["STAID"].astype(str).str.zfill(8)

# ==============================
# 4️⃣ filter stations that are in nc_station_ids
# ==============================

station_hourly = station[station["station_id"].isin(nc_station_ids)].copy()

print("Matched stations:", len(station_hourly))


# ==============================
# 5️⃣ save the filtered attributes to a new csv file
# ==============================

#save_path = "/mnt/datawaha/hyex/wkkong/data/CAMELSH/attributes/hourly_stations_3166.csv"

#station_hourly.to_csv(save_path, index=False)

#print("Saved to:", save_path)


Number of nc stations: 3166
Matched stations: 3148


In [ ]:
import pandas as pd

# transform station_id to match nc file format (zero-padded 8 digits)
station_hourly["station_id"] = station_hourly["STAID"].astype(str).str.zfill(8)

# save the filtered attributes to a new csv file
station_meta = station_hourly[["station_id", "LAT_GAGE", "LNG_GAGE"]].copy()

station_meta.head()


,station_id,LAT_GAGE,LNG_GAGE
3,03157000,39.588398,-82.578495
4,03157500,39.565066,-82.474601
5,03219500,40.419504,-83.197137
6,03220000,40.248395,-83.173803
7,03221000,40.143396,-83.120466


In [ ]:
import numpy as np

def select_nearby(meta_df, target_lat, target_lon, n):
    df = meta_df.copy()
    df["dist"] = np.sqrt(
        (df["LAT_GAGE"] - target_lat) ** 2 +
        (df["LNG_GAGE"] - target_lon) ** 2
    )
    return df.sort_values("dist").head(n)

# east
east_sites = select_nearby(station_meta, 40, -80, 10)

# west
west_sites = select_nearby(station_meta, 40, -123, 5)

selected = pd.concat([east_sites, west_sites])

selected_ids = selected["station_id"].values

print("Selected stations:")
print(selected_ids)


Selected stations:
['03072000' '03062500' '03062400' '03085500' '03085213' '03085049'
 '03084698' '03084800' '03070500' '03085956' '11473900' '11472160'
 '11525630' '11475500' '11449500']


In [ ]:
import numpy as np

# 1️⃣ filter stations west of -110 longitude
west_all = station_meta[
    station_meta["LNG_GAGE"] < -110
]

# 2️⃣ randomly select 5 stations from the west subset
west_sites = west_all.sample(5, random_state=43)

selected_ids = west_sites["station_id"].values

print("Random West stations:")
print(selected_ids)

Random West stations:
['09312800' '14198400' '06061500' '14206950' '06119600']


In [ ]:
import xarray as xr
import os

data_dir = "/mnt/datawaha/hyex/wkkong/data/CAMELSH/timeseries/Data/CAMELSH/timeseries"

ds_dict = {}

for stn in selected_ids:
    file_path = os.path.join(data_dir, f"{stn}.nc")

    if not os.path.exists(file_path):
        print(f"{stn} not found")
        continue

    ds = xr.open_dataset(file_path)

    #  (time, dynamic_forcing)
    ds_stacked = ds.to_array(dim="dynamic_forcing")

    # change dimension name
    ds_stacked = ds_stacked.rename({"DateTime": "time"})

    # reorder dimensions
    ds_stacked = ds_stacked.transpose("time", "dynamic_forcing")

    ds_dict[stn] = ds_stacked


final_ds = xr.Dataset(ds_dict)

print(final_ds)


<xarray.Dataset> Size: 193MB
Dimensions:          (time: 394488, dynamic_forcing: 12)
Coordinates:
  * time             (time) datetime64[ns] 3MB 1980-01-01 ... 2024-12-31T23:0...
  * dynamic_forcing  (dynamic_forcing) object 96B 'Tair' 'Qair' ... 'Streamflow'
Data variables:
    09312800         (time, dynamic_forcing) float64 38MB -0.29 0.003435 ... nan
    14198400         (time, dynamic_forcing) float64 38MB 4.65 ... 0.2067
    06061500         (time, dynamic_forcing) float64 38MB -1.023 ... 0.4814
    14206950         (time, dynamic_forcing) float64 38MB 9.75 0.007025 ... 1.46
    06119600         (time, dynamic_forcing) float64 38MB 0.229 ... 1.07


In [ ]:
#select west stations with no NaN 

import xarray as xr
import os
import numpy as np
import pandas as pd

data_dir = "/mnt/datawaha/hyex/wkkong/data/CAMELSH/timeseries/Data/CAMELSH/timeseries"

# 1️⃣ select west stations (longitude < -110)
west_all = station_meta[
    (station_meta["LNG_GAGE"] < -110)
]

# 2️⃣ time range for filtering
start_date = "2010-01-01"
end_date = "2020-12-31"

# 3️⃣ no nan stations
valid_west_stations = []

for stn in west_all["station_id"].values:
    file_path = os.path.join(data_dir, f"{stn}.nc")
    if not os.path.exists(file_path):
        continue

    ds = xr.open_dataset(file_path)
    ds_stacked = ds.to_array(dim="dynamic_forcing")
    ds_stacked = ds_stacked.rename({"DateTime": "time"})
    ds_stacked = ds_stacked.transpose("time", "dynamic_forcing")

    #  2023-2024
    ds_stacked = ds_stacked.sel(time=slice(start_date, end_date))

    # check NaN
    #if np.isnan(ds_stacked.values).any():
        #continue

    valid_west_stations.append(stn)

print(f"Total valid west stations in 2023-2024: {len(valid_west_stations)}")

# 4️⃣ randomly select 5 stations from the valid west stations
if len(valid_west_stations) >= 5:
    selected_west = np.random.choice(valid_west_stations, 5, replace=False)
else:
    selected_west = valid_west_stations  

print("Selected 5 west stations:")
print(selected_west)

# 5️⃣ load the selected stations into a combined xarray Dataset
ds_dict = {}
for stn in selected_west:
    file_path = os.path.join(data_dir, f"{stn}.nc")
    ds = xr.open_dataset(file_path)
    ds_stacked = ds.to_array(dim="dynamic_forcing")
    ds_stacked = ds_stacked.rename({"DateTime": "time"})
    ds_stacked = ds_stacked.transpose("time", "dynamic_forcing")
    ds_stacked = ds_stacked.sel(time=slice(start_date, end_date))
    ds_dict[stn] = ds_stacked

final_ds = xr.Dataset(ds_dict)
print(final_ds)

Total valid west stations in 2023-2024: 0
Selected 5 west stations:
[]
<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*


In [17]:
final_ds

<xarray.Dataset> Size: 9MB
Dimensions:          (time: 17544, dynamic_forcing: 12)
Coordinates:
  * time             (time) datetime64[ns] 140kB 2023-01-01 ... 2024-12-31T23...
  * dynamic_forcing  (dynamic_forcing) object 96B 'Tair' 'Qair' ... 'Streamflow'
Data variables:
    11173510         (time, dynamic_forcing) float64 2MB 8.253 ... 1.025
    11073300         (time, dynamic_forcing) float64 2MB 10.93 ... 0.000354
    14142500         (time, dynamic_forcing) float64 2MB 1.837 ... 148.0
    11522500         (time, dynamic_forcing) float64 2MB 2.335 ... 293.8
    05014500         (time, dynamic_forcing) float64 2MB -4.85 ... 0.7277

In [ ]:
save_path = "/home/kongw0a/code/selected_stations_westnonan.nc"

#  netCDF4 
final_ds.to_netcdf(save_path, engine="netcdf4")

print("Saved final Dataset to:", save_path)

Saved final Dataset to: /home/kongw0a/code/selected_stations_westnonan.nc
